In [1]:
import pandas as pd
import re
import os

In [2]:
tab_file = pd.read_csv('./example/modified_example.tab',sep='\t',header=None)
tab_file.columns = ['Family_HMM','HMM_length','Query_ID','Query_length','E_value',
                    'HMM_start','HMM_end','Query_start','Query_end','Coverage']
tab_file.head()

,Family_HMM,HMM_length,Query_ID,Query_length,E_value,HMM_start,HMM_end,Query_start,Query_end,Coverage
0,GH76.hmm,358,gene_10003,416,9.000000e-34,30,328,74,383,0.832402
1,GH43.hmm,248,gene_10152,932,1.000000e-38,2,244,481,792,0.975806
2,CE6.hmm,99,gene_10152,932,3.600000e-07,2,95,164,268,0.939394
3,CBM6.hmm,138,gene_10152,932,9.200000e-17,16,138,817,932,0.884058
4,GH2.hmm,752,gene_10169,1031,2.700000e-221,17,750,35,912,0.974734


In [3]:
if os.path.exists('./example/modified_example.faa'):
    os.remove('./example/modified_example.faa')

with open('./example/example.faa','r') as f1:
    with open('./example/modified_example.faa','a') as f2:
        i = 1
        line = f1.readline()
        while(line):       
            if line[0] == '>':    #start with '>'
                #print('That is the header!')
                f2.write(">gene_%d\n"%i)   #re-write the header
                #print(">gene_%d\n"%i)
                i += 1
            else:
                f2.write(line)
                #print(line)
            line = f1.readline()

In [4]:
with open('./example/modified_example.faa','r') as f:
    seq_file = f.read()

In [5]:
cazy_tab = pd.read_csv('cazy_level_tab.txt',sep='\t',header=0)
cazy_tab.head()

,L1,L2,L3,L4
0,CAZyme,Enzyme_Classes,GlycosylTransferases,GT
1,CAZyme,Enzyme_Classes,GlycosideHydrolases,GH
2,CAZyme,Enzyme_Classes,CarbohydrateEsterases,CE
3,CAZyme,Enzyme_Classes,PolysaccharideLyases,PL
4,CAZyme,Enzyme_Classes,AuxiliaryActivities,AA


In [6]:
CAZYme_family = {'CAZyme':{'Enzyme_Classes':{'GlycosylTransferases':'GT',
              'GlycosideHydrolases':'GH',
              'CarbohydrateEsterases':'CE',
              'PolysaccharideLyases':'PL',
              'AuxiliaryActivities':'AA'},
              'Associated_Modules':{'Carbohydrate-BindingModules':'CBM'}}}

In [7]:
def find_value_recursive(cazy_dict,cazyme):
    global flag
    global levels
    flag = 0
    levels = cazyme
    for key,value in cazy_dict.items():
        if isinstance(value,dict):
            global n
            n += 1
            find_value_recursive(value, cazyme)
            if flag == 1:
                n -= 1
                levels =  'L'+str(n)+'_'+key+';'+levels
                return levels
            else:
                n -= 1
        elif cazyme == value:
            n += 1
            flag = 1
            levels =  'L'+str(n)+'_'+key+';'+'L'+str(n+1)+'_'+levels
            global n_level
            n_level = n+1
            return


In [13]:
if os.path.exists('./example/modified_example.fasta'):
    os.remove('./example/modified_example.fasta')
    
if os.path.exists('./example/modified_example_tax.txt'):
    os.remove('./example/modified_example_tax.txt')
        
        
with open('./example/modified_example.fasta','a') as f1:
    for i in range(0,tab_file.shape[0]):
        #global n,n_level
        #n = 0
        #n_level = 0
        family = tab_file.loc[i,'Family_HMM'].replace('.hmm','')
        query_id = tab_file.loc[i,'Query_ID']
        query_start = tab_file.loc[i,'Query_start']
        query_end = tab_file.loc[i,'Query_end']
        query_length = tab_file.loc[i,'Query_length']
        #print('%d: %s'% (i+1,family))
        seq_pattern = re.compile(r'>'+query_id+'\n(.*?)(>|\n$)',re.S) #we cannot replace . with [A-Z]
        try:
            query_sequence = re.search(seq_pattern,seq_file).group(1).replace('\n','') #find the sequence. |n will be counted as char.
        except:
            print('Did not find %s in sequence!' % family)
            continue
        if(len(query_sequence)!=query_length): #check the length
            print("The length of %s did not match! Please check the input seqs!" % family)
            continue
        
        #print('Query Sequence: %s'%query_sequence)
        target_sequence = query_sequence[query_start-1:query_end] #extract the sequence
        #print('Target Seuqunce: %s\n'%target_sequence)

        f1.write('>cazy_%04d_%s\n%s\n'%(i,query_id,target_sequence))

with open('./example/modified_example_tax.txt','a') as f2:
    for i in range(0,tab_file.shape[0]):
        name_pattern = re.compile(r'(GH|GT|CBM|PL|AA|CE)(\d+)(_\d+)?') #find the cazyme category with regular expression
        query_id = tab_file.loc[i,'Query_ID']
        family = tab_file.loc[i,'Family_HMM'].replace('.hmm','')
        try:
            cazy_cat = re.search(name_pattern,family).group(1)
            cazy_num = re.search(name_pattern,family).group(2)
        except:
            print('Find %s! What is that?' % family) #if we found something interesting
            continue   #Shall we continue?
            
        '''
        # method 1   recursive way 
        cazy_prefix = find_value_recursive(CAZYme_family,cazy_cat)
        cazy_tax = cazy_prefix+cazy_num
        if cazy_cat == 'GH' and int(cazy_num) in [5,13,30,43]:
            try:
                cazy_sub_num = re.search(name_pattern,family).group(3)
            except:
                print('%s%s had no sub family.'%(cazy_cat,cazy_num))
            cazy_tax = cazy_tax+';L'+str(n_level+1)+'_'+family
        '''
        
        # method 2
        temp = cazy_tab.loc[cazy_tab.loc[:,'L4']==cazy_cat,:]
        cazy_tax = ''
        for j in range(0, cazy_tab.shape[1]):
            cazy_tax = cazy_tax + temp.columns[j] + '_' + temp.iloc[0, j] + ';' #join the headers(levels) of cazyme.
        
        cazy_tax = cazy_tax.strip(';') + cazy_num
        if cazy_cat == 'GH' and int(cazy_num) in [5,13,30,43]:
            if re.search(name_pattern,family).group(3): #find the subfamily
                cazy_tax = cazy_tax+';L5_'+family
            #try:
            #    cazy_sub_num = re.search(name_pattern,family).group(3)
            #    cazy_tax = cazy_tax+';L5_'+family
            #except:
            #    print('%s%s had no sub family.'%(cazy_cat,cazy_num))
        #print(cazy_tax)
        #print('\n')
        f2.write('cazy_%04d_%s\t%s\n'%(i,query_id,cazy_tax))  


Find SLH! What is that?
Find SLH! What is that?
Find dockerin! What is that?
Find dockerin! What is that?
Find dockerin! What is that?
Find SLH! What is that?
Find dockerin! What is that?
Find dockerin! What is that?
Find dockerin! What is that?
Find dockerin! What is that?
Find cohesin! What is that?
Find dockerin! What is that?
Find SLH! What is that?
Find cohesin! What is that?
Find SLH! What is that?
Find SLH! What is that?
Find dockerin! What is that?
Find dockerin! What is that?
Find dockerin! What is that?
